<a href="https://colab.research.google.com/github/LiamAMcNamara/HMMs-Lab3/blob/master/McNamara_Lab_3_HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Lab #3: Hidden Markov Models**

Liam McNamara.308

June 11, 2020

---
---

**Part 1 (50 Points):** Implement the Viterbi Algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten). See elsner.hmm.xls for a bit more discussion on the problem. Remeber that the Viterbi algorithm computes the most likely sequence for an input.

Your solution should be able to handle variable length sequences (in the range of 3-5).

There are 3 files provided for you that have observation/emission probabilities, transition probabilities, and test sequence data (for evaluation), respectively.

Please read the probabilities and observations from their Carmen URLs, *do not hard-code the values*. Refer back to lab 2 for exampes of working with CSV files and how to download them from URLs.

The observation and transition probabilities have rows being the variable of interest, and columns being the conditioning variables. For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label). The columns sum up to 1.

The test data has one line per sequence. When a sequence is less than five observations long, the last  columns are filled with zeros.



Set up the environment

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random as rdm

Read in the provided data from Carmen URL

In [2]:

emissionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/observationProbsLecture.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/transitionProbsLecture.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/testDataLecture.csv'
testData_dataframe = pd.read_csv(testDataURL)

Above is the data from the example we did in lecture (and checkpoint exercise) converted to CSV format. It can be run in place of the ice cream data for part 1, but not for part 2. One will observe the same M and C-matricies as well as most likely sequence values. The purpose for this was to check my results. In order for it to work I had to write my code for part 1 in a way that accomodated data with 3 hidden vairables. I believe this process was worthwhile.

In [4]:
emissionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv'
testData_dataframe = pd.read_csv(testDataURL)

Print out the top of the dataframes to make sure the data loaded correctly. For the Ice Cream problem, the data tables should have the following format:

Emission Probabilities [3, 3]: 3 columns (P(x|...), C, H), and 3 rows corresponding the the number of ice creams

Transition Probabilities [3, 4]: 4 columns (P(x|...), C, H, START), and 3 rows (C, H, STOP)

Test Sequence Data [10, 6]: 6 columns (SeqNumber, Obs1...5), and 10 rows corresponing the the sequence sample number.

Note: The test data has one line per sequence. When a sequence is less than five observations long, the last columns are filled with zeros.

In [5]:
print("Emission Probabilities")
display(emission_dataframe.shape)
display(emission_dataframe.head())
print('\n')
print("Transition Probabilities")
display(transition_dataframe.shape)
display(transition_dataframe.head())
print('\n')
print("Test Sequence Data")
display(testData_dataframe.shape)
display(testData_dataframe.head())

Emission Probabilities


(3, 3)

,P(x|...),C,H
0,1,0.6407,0.0002
1,2,0.1481,0.5341
2,3,0.2122,0.4657




Transition Probabilities


(3, 4)

,P(x|...),C,H,START
0,C,0.87,0.47,0.5
1,H,0.07,0.47,0.5
2,STOP,0.06,0.06,0.0




Test Sequence Data


(10, 6)

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,1,2,3,3,2,3
1,2,2,3,2,2,0
2,3,3,1,3,3,1
3,4,2,1,1,0,0
4,5,1,1,1,2,3


Use A and B shorthand for transition and emission probabilities respectively

In [31]:
# In our class notes we referred to the transition matrix as 'A'
# and the emission/observation matrix as 'B'. So I will be doing the same.
A = transition_dataframe.to_numpy()
A = np.delete(A, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (C, H, STOP)
B = np.array(emission_dataframe)
B = np.delete(B, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (1, 2 , 3)

sequences = testData_dataframe.to_numpy()
sequences = np.delete(sequences, 0, 1)  # remove extra (label) column included in
                                        # the df. This column inc. (1 ... n seqs)

print(f'A:\n{A}')
print(f'B:\n{B}')
print(sequences)

A:
[[0.87 0.47 0.5]
 [0.07 0.47 0.5]
 [0.06 0.06 0.0]]
B:
[[6.407e-01 2.000e-04]
 [1.481e-01 5.341e-01]
 [2.122e-01 4.657e-01]]
[[2 3 3 2 3]
 [2 3 2 2 0]
 [3 1 3 3 1]
 [2 1 1 0 0]
 [1 1 1 2 3]
 [1 3 1 1 0]
 [3 2 3 0 0]
 [2 2 1 1 0]
 [1 3 2 3 3]
 [1 1 1 0 0]]


Find most likely sequence of hidden varialbles given the obervation sequence.

Below you will see outputs of the M and C-Matrix. For the C-Matrix, I incremented the values for printing so that they match the notation we used in lecture. However in my implementation, what prints as [1, 2, 1] is actually [0,  1, 0]. This is so that I can use the values in the C-matrix as indexes in the backward-phase. The same goes for the most likely sequence.

In [7]:
# function to pad an array to the same length of the test sequence data (# of observations)
def pad(arr):
  while len(arr) != testData_dataframe.shape[1]-1:  # subract one for extra col label
    arr = np.append(arr, 0)
  return arr

In [32]:
# array to store most likely sequences (Used for part 2)
mostLikelySequences = np.zeros_like(sequences)
for n in range(len(sequences)): 
  # remove one sequence from matrix of sequences
  seq = sequences[n]
  seq = seq[seq != 0] # trim the sequence of zeros, since these are not obsevations
  print(f'Observed sequence {n+1}: {seq}\n')

  #Initialize M and C-Matricies for the correct shape given the sequence n
  # M.shape = [sequence n length, # of hidden variables]
  # In ice cream case, Sequence of length 3-5 with 2 possible hidden vairable values (H, C)
  m_matrix = np.zeros((len(seq), emission_dataframe.shape[1]-1)) # trim pesky label column

  # C.shape = [sequence length - 1, # hidden variables]
  # shape of C is the same as the M-Matrix minus one row, transposed.
  c_matrix = np.delete(m_matrix, 0, 0)
  c_matrix = c_matrix.transpose()

  seq = seq - 1 # decrement elements of seq for proper indexing
  
  # Do viterbi algorithm... returns the most likely sequence for the given seq
  mls = forward(m_matrix, c_matrix)
  # Store most likely sequences for use in part 2. (requires some manipulation)
  mls = mls + 1 # incrememnt for proper printing before padding
  mls = pad(mls)  # pad to give each most likely sequence a length of 5
  mostLikelySequences[n] = mls

#print(mostLikelySequences)

Observed sequence 1: [2 3 3 2 3]

Forward phase calculations:
M[0][0]: 0.5 * 0.1481
M[0][1]: 0.5 * 0.5341
M[2][1]: 0.2122 * 0.18886983289357961
M[2][2]: 0.4657 * 0.36796687188507765
M[3][1]: 0.2122 * 0.1649071012306746
M[3][2]: 0.4657 * 0.3809122556569919
M[4][1]: 0.1481 * 0.14334479886912377
M[4][2]: 0.5341 * 0.39256085578334693
M[5][1]: 0.2122 * 0.07999072256804153
M[5][2]: 0.4657 * 0.4267866211414028
M-Matrix:
[[0.21709176 0.78290824]
 [0.18954839 0.81045161]
 [0.16476414 0.83523586]
 [0.09194336 0.90805664]
 [0.07868236 0.92131764]]
C-Matrix (transposed):
[[1. 2.]
 [1. 2.]
 [1. 2.]
 [1. 2.]]
Most Likely Sequence: [2 2 2 2 2]
___________________________________________________________

Observed sequence 2: [2 3 2 2]

Forward phase calculations:
M[0][0]: 0.5 * 0.1481
M[0][1]: 0.5 * 0.5341
M[2][1]: 0.2122 * 0.18886983289357961
M[2][2]: 0.4657 * 0.36796687188507765
M[3][1]: 0.1481 * 0.1649071012306746
M[3][2]: 0.5341 * 0.3809122556569919
M[4][1]: 0.1481 * 0.0932460344410975
M[4][2]: 0.

**Viterbi Algorithm**

Forward Phase: Calculate values for the M-Matrix. One step at a time. Starting at first state and moving forward. Filling in the C-Marix as we go.

Choose value for last state by maximizing value of hidden state. I print some of the more important calculations out so that it is clear what is going on.

In [8]:
def forward(m_matrix, c_matrix):
  # use priors to set first row of M matrix
  print("Forward phase calculations:")
  for i in range(B.shape[1]):
    m_matrix[0,i] = A[i,A.shape[1]-1] * B[seq[0], i]
    print(f'M[0][{i}]: {A[i,A.shape[1]-1]} * {B[seq[0], i]}')

  normalizeRow(m_matrix, 0); # normalize each row as we calculate it to prevent underflow
  vect = np.zeros(3)
  for s in range(1, len(seq)): # each element (besides first) of the sequence
    for k in range(B.shape[1]): # k = weather today
      for j in range(B.shape[1]): # j = weather yesterday
        # Store the hypothetical probabilities of yesterdays weather to be compared
        vect[j] = (A[j][k] * m_matrix[s-1][j])
      # maximize probability
      max = np.max(vect)
      print(f'M[{s+1}][{k+1}]: {B[seq[s], k]} * {max}')
      # store 'magic' value in M-Matrix.
      m_matrix[s][k] = B[seq[s], k] * max
      # store our (numerical) weather choice  in C
      c = np.where(vect == max)
      c_matrix[k][s-1]=c[0][0]
    normalizeRow(m_matrix, s) # normalize each row as we calculate it to prevent underflow

  c_matrix_print = c_matrix + 1 # increment C Matrix for proper printing
  print(f'M-Matrix:\n{m_matrix}')
  print(f'C-Matrix (transposed):\n{c_matrix_print.transpose()}')  # Un-Transpose C matrix for printing
  # convert C-Matrix to int so we can use its values as indexes
  c_matrix = c_matrix.astype(int)
  # Start backward phase of viterbi algorithm after forward is complete
  MLS = backward(m_matrix, c_matrix)
  return MLS

**Backward Phase**: Use final row of M-matrix to find final member of the most likely sequence. From this, us C-Matrix to work backwards and find what the values of the preceding hidden states should be.

In [9]:
def backward(m_matrix, c_matrix):
  MLS = np.zeros_like(seq)  # intialize Most Likely Sequence (same length as test sequence)
  # use M-Matrix to find the value of final hidden variable
  last_row = m_matrix[len(seq)-1]
  m_max = np.max(last_row)
  last_hidden = np.where(last_row == m_max)
  MLS[len(seq)-1] = last_hidden[0][0]

  # use the C matrix to find the value of the remaining hidden variables
  for i in range(len(seq) - 2, -1, -1):
    x = c_matrix[MLS[i+1]][i]
    MLS[i] = x
  MLS_print = MLS + 1
  print(f'Most Likely Sequence: {MLS_print}')
  print('___________________________________________________________\n')
  return MLS

In [10]:
# function to normalize a specific row in a matrix
def normalizeRow(m, i):
  # Normalize each row in M-Matrix (each obseervation) to prevent underflow
  norm_m = m  #python uses pass by reference so this will change m
  row = m[i]
  row = row / np.sum(row)
  norm_m[i] = row


**Part 2 (50 points)**: Using the same network, implement a likelihood samples for approximate inference. For any test sequence, sample complete sequences of the hidden states *n* times, where *n* can range from 10 to 100,000 samples. The goal is to approximate the likelihood of *all possible* sequences.

Assuming the Viterbi sequence is "correct", how long (how many samples) does it take the sampler to converge so that the (estimated) most likely sample you get matches the Viterbi sequence?

---
---


Diregard STOP probabilities and normalize remaining transition probabilities

In [17]:
#remove STOP probabilities (bottom row)
transitionProbs = np.delete(A, A.shape[0]-1, 0)
#remove and store START priors (right-most column)
priors = transitionProbs[:,A.shape[1]-1]
transitionProbs = np.delete(transitionProbs, transitionProbs.shape[1]-1, 1)
#normalize across columns
normalizeCol(transitionProbs, 0)
normalizeCol(transitionProbs, 1)
print(f'transitionProbs:\n{transitionProbs}')
print(f'B (Emission Probabilities):\n{B}')

transitionProbs:
[[0.9255319148936171 0.5]
 [0.07446808510638299 0.5]]
B (Emission Probabilities):
[[6.407e-01 2.000e-04]
 [1.481e-01 5.341e-01]
 [2.122e-01 4.657e-01]]


In [14]:
# function to normalize a column in a matrix
def normalizeCol(m, i):
  # Normalize each row in M-Matrix (each obseervation) to prevent underflow
  norm_m = m  #python uses pass by reference so this will change m
  col = m[:,i]
  col = col / np.sum(col)
  norm_m[:,i] = col


Approximae inference by likelihood waiting. Sample each hidden state a number of times.

In [54]:
for n in range(len(sequences)):
  # retreive and manipulate testData sequences
  seqt = sequences[n]
  seqt = seqt[seqt != 0]
  print(f'Observed sequence {n+1}: {seqt}')
  seqt = seqt - 1 # for indexing

  # retreive and manipulate most likely sequences from part 1
  mls_viterbi = mostLikelySequences[n]
  mls_viterbi = mls_viterbi[mls_viterbi != 0]
  mls_viterbi = mls_viterbi - 1 # for consistency of indexing
  # initialize sample sequence with same number of zeros as test seq
  sampseq = np.zeros_like(seqt)
  mlsw = np.zeros_like(seqt)
  bestweight = 0
  its = 0
  for i in range(100000):
    weight = weightedInference(seqt, sampseq)
    # keep track of most likely (highest weighted) sample sequence generated
    if weight > bestweight:
      # print(f'new best[{i}]: {sampseq + 1} weight: {weight}')
      mlsw = np.copy(sampseq)
      bestweight = weight
    sampseq_print = sampseq + 1
    its = i
    # compare to known most likely sequence, break if equal
    if i > 9:
      if np.array_equal(mlsw, mls_viterbi):
        its = i
        break
      
  print()
  if (its == 99999):
    print(f'After {its + 1} iteration(s), the sampled sequence did not converge to match the Viterbi sequence.')
    print("Instead, it converged to the following:")
  else:
    print(f'Sampled sequence converged to match Viterbi sequence after {its} iteration(s)')
  mlsw_print = mlsw + 1 # for printing
  print(f'Most Likely Weighted Sample Sequence: {mlsw_print}')
  print(f'Viterbi sequence {n+1} (for reference): {mls_viterbi + 1}')
  print(f'Highest Weight: {bestweight}\n')
  print("_________________________________")
  

Observed sequence 1: [2 3 3 2 3]

Sampled sequence converged to match Viterbi sequence after 10 iteration(s)
Most Likely Weighted Sample Sequence: [2 2 2 2 2]
Viterbi sequence 1 (for reference): [2 2 2 2 2]
Highest Weight: 0.028811367344428896

_________________________________
Observed sequence 2: [2 3 2 2]

Sampled sequence converged to match Viterbi sequence after 10 iteration(s)
Most Likely Weighted Sample Sequence: [2 2 2 2]
Viterbi sequence 2 (for reference): [2 2 2 2]
Highest Weight: 0.07095352427853971

_________________________________
Observed sequence 3: [3 1 3 3 1]

Sampled sequence converged to match Viterbi sequence after 10 iteration(s)
Most Likely Weighted Sample Sequence: [1 1 1 1 1]
Viterbi sequence 3 (for reference): [1 1 1 1 1]
Highest Weight: 0.003922343159133334

_________________________________
Observed sequence 4: [2 1 1]

Sampled sequence converged to match Viterbi sequence after 10 iteration(s)
Most Likely Weighted Sample Sequence: [1 1 1]
Viterbi sequence 4 

In [52]:
def weightedInference(seqt, sampseq): # NOTE: (for calulation) 0 = C, 1 = H
  # sample first hidden state using priors
  x = rdm.random()
  if x < 0.5: sampseq[0] = 0
  else: sampseq[0] = 1
  weight = B[seqt[0]][sampseq[0]]

  # sample remaining states using transition probabilities
  for i in range(1, len(seqt)):
    x = rdm.random()
    if x < transitionProbs[sampseq[i-1], 0]: sampseq[i] = 0
    else: sampseq[i] = 1

    weight *= B[seqt[i]][sampseq[i]]
  
  return weight

**Bonus (25 Points):** Implement the Forward-Backward algorithm. Test the algorithm on the Elnser dataset for both 2 hidden states (hot, cold) and 3 hidden states (hot, warm, cold). Report on the differences that you see.

---
---